In [309]:
import sqlite3 as db 
import pandas as pd
conn = db.connect('soccer/database.sqlite')

In [310]:
print(conn)

In [311]:

def get_all_players_info(year):

    
    query_str = """select  p.player_name,pa2.* from player_attributes pa2, player p where p.player_api_id = pa2.player_api_id and
    (pa2.player_api_id, pa2.date) in (
    select z.player_api_id, z.date from (
    select pa.player_api_id, pa.date, min(abs(strftime('%%s',pa.date) - strftime('%%s','%(year)s-09-01 00:00:00'))) from player_attributes pa, player p 
                                    where pa.player_api_id = p.player_api_id
                                    and strftime('%%s',pa.date) > strftime('%%s','2014-05-14 00:00:00') 
                                    and pa.date like '%(year)s%%'
                                    group by pa.player_api_id ) z )"""%{'year':year}
    
    all_player_attrs = pd.read_sql_query(query_str,conn)
    
    return all_player_attrs


In [312]:

year_data = {'2014': 'statbunker-football-stats/Player Stats 2014-15.csv' , 
             '2015': 'statbunker-football-stats/Player Stats 2015-16.csv' ,
             '2016': 'statbunker-football-stats/Player Stats 2016-17.csv'}


In [313]:
def norm_player_names(all_players_info):
    all_players_info.loc[lambda df: df['Team'] == 'Manchester United', 'Team'] = 'Man United'
    
    return all_players_info

In [318]:

def create_team( name, year):
    
    ptm = pd.read_csv(year_data[year] , dtype = {'overall_rating':float} )
   
    ptm = ptm[(ptm['Type Of Goal'] == 'Player') & (ptm['League'] == 'Premier League')]
    
    all_players_info  = get_all_players_info(year)
        
    joined = all_players_info.set_index('player_name').join(ptm.set_index('Player'), how='inner')
    
    
    full_team = joined[joined['Team'].str.startswith(name)]
        
    pos_sel = {'Defender': 4, 'Midfielder': 3, 'Forward': 3, 'Goalkeeper':1}
    team_agg = {}
    
    team_df = {}
    
    df_start = full_team[full_team['Team'] == 'THIS DOES NOT EXIST']
    
    for key,value in pos_sel.items():
        
        grp = full_team[full_team['Position'] == key].sort_values(by = ['overall_rating'], ascending = False).head(value)
        df_start = df_start.append(grp)
        avg = grp.mean()
        team_agg[key] = avg
    
    return df_start

In [319]:

def agg_attributes(team):
    #team[team['Position'] == 'Goalkeeper']['Position'] = 'Defender'
    
    team.loc[lambda df: df['Position'] == 'Goalkeeper', 'Position'] = 'Defender'
       
    
    g = team.groupby(['Position'] , as_index=False).mean()
    #print(g['overall_rating'])
    #g = g.reset_index().set_index('index')
    
    return g.transpose()

In [320]:
def get_fixtures(year):
    
    f = pd.read_csv('epl-results-19932018/EPL_Set.csv')
    return f[f['Season'] == year]
    
    

In [321]:
# team = create_team('Liverpool','2014')

# team_attrs = agg_attributes(team)
# team_attrs.columns = ['Defense','Attack','Midfield']
# team_attrs

team = create_team('Leicester','2014')

f = get_fixtures('2014-15').head(10)


res_df = pd.DataFrame()

for key,row in f.iterrows():
    print("MATCHING UP: %s vs %s"%(row['HomeTeam'], row['AwayTeam']))
    home = row['HomeTeam']
    away = row['AwayTeam']
    #print(home,away)
    home_team = create_team(home,'2014')
    away_team = create_team(away,'2014')
    home_attrs = agg_attributes(home_team)
    away_attrs = agg_attributes(away_team)
    
    
    norm_attrs = {}
    
#     print(home_attrs, away_attrs)
    print("Overalls: ", home_attrs[0]['overall_rating'],away_attrs[0]['overall_rating'])
    
    norm_attrs['Def_Score'] = home_attrs[0]['overall_rating'] - away_attrs[1]['overall_rating']
    norm_attrs['Mid_Score'] = home_attrs[2]['overall_rating'] - away_attrs[2]['overall_rating']
    norm_attrs['Att_Score'] = home_attrs[1]['overall_rating'] - away_attrs[0]['overall_rating']
    norm_attrs['Result'] = 1 if row['FTHG'] - row['FTAG'] > 0  else (-1 if row['FTHG'] - row['FTAG'] < 0 else 0)
    
    
    
#     print(row)
#     print(norm_attrs)
#     norm_df = pd.DataFrame.from_dict(norm_attrs, orient='     ')
#     res_df = res_df.append(norm_df)
    
   # print(home_attrs[0]['overall_rating'])
    
    #print(home_attrs[home_attrs['index'] == 'overall_rating'])
    
    #print(len(home_team),len(away_team))
    
   # print(row)


KeyError: 'Season'